# Datetime datasus

## Import data

In [0]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [8]:

sinasc_1 = pd.read_csv('http://bit.ly/df_sinasc_es_2016', encoding='latin-1')
sinasc_2 = pd.read_csv('http://bit.ly/df_sinasc_es_2015', encoding='latin-1')
sinasc_3 = pd.read_csv('http://bit.ly/df_sinasc_es_2014', encoding='latin-1')
sinasc_4 = pd.read_csv('http://bit.ly/df_sinasc_es_2013', encoding='latin-1')
sinasc_5 = pd.read_csv('http://bit.ly/df_sinasc_es_2012', encoding='latin-1')
sinasc = pd.concat([sinasc_1, sinasc_2, sinasc_3, sinasc_4, sinasc_5])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [10]:
sim_1 = pd.read_csv('http://bit.ly/df_sim_es_2016', encoding='latin-1')
sim_2 = pd.read_csv('http://bit.ly/df_sim_es_2015', encoding='latin-1')
sim_3 = pd.read_csv('http://bit.ly/df_sim_es_2014', encoding='latin-1')
sim_4 = pd.read_csv('http://bit.ly/df_sim_es_2013', encoding='latin-1')
sim_5 = pd.read_csv('http://bit.ly/df_sim_es_2012', encoding='latin-1')
sim = pd.concat([sim_1, sim_2, sim_3, sim_4, sim_5])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [11]:
sinasc.shape, sim.shape

((273802, 71), (110497, 98))

## Integração dos dados

In [0]:
dndo_ndn_inner = pd.merge(sinasc, sim, how='inner', on='NUMERODN',  indicator='merge')

In [13]:
dndo_ndn_inner.shape

(1438, 169)

In [0]:
dndo_ndn_inner['MORTALIDADE'] = 1
sinasc['MORTALIDADE'] = 0

In [17]:
dndo = pd.concat([dndo_ndn_inner, sinasc])
dndo.drop_duplicates(subset='NUMERODN', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
dndo.MORTALIDADE.value_counts()

0    272369
1      1433
Name: MORTALIDADE, dtype: int64

## Delta para idade

In [0]:
def convert(data):
    d = int(data/1000000)
    data = data - d*1000000
    m = int(data/10000)
    data = data - m*10000
    y = data
    fin = y*10000 + m*100 +d
    return fin

In [0]:
do = dndo[dndo['MORTALIDADE'] == 1]
dn = dndo[dndo['MORTALIDADE'] == 0]

In [21]:
do.DTNASC_x = do.DTNASC_x.apply(convert)
do.DTNASC_x = pd.to_datetime(do.DTNASC_x, format='%Y%m%d')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
do.DTOBITO = do.DTOBITO.apply(convert)
do.DTOBITO = pd.to_datetime(do.DTOBITO, format='%Y%m%d')


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [23]:
do['age'] = -1
do['age'] = (do.DTOBITO - do.DTNASC_x).dt.total_seconds()/86400 #total seconds of day
do.age.describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    1433.000000
mean       36.452896
std        81.162749
min       -10.000000
25%         1.000000
50%         4.000000
75%        30.000000
max      1526.000000
Name: age, dtype: float64

In [24]:
do.shape

(1433, 204)

In [0]:
do_menor = do[do['age'] < 365]
do_menor = do[do['age'] > 0]

In [31]:
do[do['age'] < 0].DTNASC_x, do[do['age'] < 0].DTOBITO

(1287   2012-05-29
 Name: DTNASC_x, dtype: datetime64[ns], 1287   2012-05-19
 Name: DTOBITO, dtype: datetime64[ns])